## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
import os
import keras
import itertools
from keras.layers import BatchNormalization, Activation

In [11]:
pip install Resnet

  Created wheel for Resnet: filename=resnet-0.1-py3-none-any.whl size=10046 sha256=438ce8ae65c72afe910fcb2a7b81449e68050173da2463bcc0386d1879941fb8
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\3c\db\ed\d285a3be8fa331aca2474dbd87b0e6c2ce5a9060f836723464
Successfully built Resnet
Note: you may need to restart the kernel to use updated packages.


In [13]:
from keras.datasets import cifar10
from resnet_builder import resnet
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

In [14]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [16]:

batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
782/782 [==============================] - 39s 36ms/step - loss: 2.1145 - accuracy: 0.4270 - val_loss: 2.5838 - val_accuracy: 0.3516
Epoch 2/30
782/782 [==============================] - 29s 37ms/step - loss: 1.3935 - accuracy: 0.6431 - val_loss: 1.5199 - val_accuracy: 0.5624
Epoch 3/30
782/782 [==============================] - 27s 35ms/step - loss: 1.1490 - accuracy: 0.7165 - val_loss: 1.4071 - val_accuracy: 0.6249
Epoch 4/30
782/782 [==============================] - 27s 34ms/step - loss: 0.9785 - accuracy: 0.7701 - val_loss: 1.3220 - val_accuracy: 0.6496
Epoch 5/30
782/782 [==============================] - 27s 34ms/step - loss: 0.8771 - accuracy: 0.8017 - val_loss: 1.4007 - val_accuracy: 0.6607
Epoch 6/30
782/782 [==============================] - 26s 34ms/step - loss: 0.7936 - accuracy: 0.8299 - val_loss: 1.2361 - val_accuracy: 0.6910
Epoch 7/30
782/782 [==============================] - 27s 34ms/step - loss: 0.7342 - accuracy: 0.8516 - val_loss: 1.6961 - val_accuracy: